In [46]:
pip install azure-search-documents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [85]:
#import libraries
import textwrap
import os
import re
import torch
from transformers import BertModel, BertTokenizer
import json
import pandas as pd

In [6]:
csv_file = 'all_meeting_merged.csv'

In [7]:
df = pd.read_csv(csv_file)
df.head()

,Id,Session,Meeting,Speaker,Text
0,S1M11,1,1,﻿Chairman,Excellencies. I am pleased to declare the firs...
1,S1M12,1,1,Chile,Before going to the logistics and the backgrou...
2,S1M13,1,1,Russia,"Thank you, Mr. President. I will now ask for t..."
3,S1M14,1,1,Chairman,Thank you for your attention. I thank the repr...
4,S1M15,1,1,Cuba,"Cuba representative, please. Thank you, Presid..."


In [8]:
df.tail()

,Id,Session,Meeting,Speaker,Text
1264,S3M774,3,7,Chairman,I thank the distinguished representative of th...
1265,S3M775,3,7,Russia,"Thank you very much, Chair. At the outset, Ple..."
1266,S3M776,3,7,Chairman,I thank the distinguished representative of th...
1267,S3M777,3,7,France,"Mercy, Mr. President. Thank you, Chair. I'd li..."
1268,S3M778,3,7,Chairman,I think the distinguished representative of Fr...


In [9]:
# Get row where 'Speaker' column contains the specified string
df = df[df['Speaker'] != 'thank you, Mr. President, and thank you, dear German delegate, for this question. I hope I understood it correctly. To what extent, as far as I understand, you asked this question. The Chekhov Convention and other agreements have a position that interact and that rely on each other or are restricted from each other. This is related to understanding the articles in this agreement. I am assuming that... in order to... well, in the space agreement it is based on the usual international law and many positions coincide with the positions of other agreements and agreements. But it is a little more limited. And of course, as it is the Chekhov Convention, the right to space is based on the Chekhov Convention. Chairman']


In [10]:
#chunk text
def chunk_text(row):
    text = row['Text']
    if len(text) > 2500:
        return textwrap.wrap(text, 2500)
    else:
        return [text]



In [11]:
df_chunks = df
df_chunks.head()

,Id,Session,Meeting,Speaker,Text
0,S1M11,1,1,﻿Chairman,Excellencies. I am pleased to declare the firs...
1,S1M12,1,1,Chile,Before going to the logistics and the backgrou...
2,S1M13,1,1,Russia,"Thank you, Mr. President. I will now ask for t..."
3,S1M14,1,1,Chairman,Thank you for your attention. I thank the repr...
4,S1M15,1,1,Cuba,"Cuba representative, please. Thank you, Presid..."


In [12]:
# Retrieve the first row of df in a DataFrame
row = df_chunks.iloc[0]
text = row['Text']
id = row['Id']
session = row['Session']
meeting = row['Meeting']
speaker = row['Speaker']
chunks = chunk_text(row)

# Print id, session, meeting, speaker, and chunks on new lines
print('Id:', id)
print('Session:', session)
print('Meeting:', meeting)
print('Speaker:', speaker)
print('Text:', text)
print('Chunks:', chunks)

Id: S1M11
Session: 1
Meeting: 1
Speaker: ﻿Chairman
Text: Excellencies. I am pleased to declare the first session of the non-limited composition work group on the reduction of space threats through rules and principles of responsible behaviour.
Chunks: ['Excellencies. I am pleased to declare the first session of the non-limited composition work group on the reduction of space threats through rules and principles of responsible behaviour.']


In [13]:
#create a df where text is each chunk
row = df_chunks.iloc[0]
chunks = chunks = chunk_text(row)
session = session * len(chunks)
meeting = meeting * len(chunks)
speaker = speaker * len(chunks)
chunk_ids = []
i= 1
for chunk in chunks:
    chunk_id = f'{id}{i}'
    i+=1
    chunk_ids.append(chunk_id)

df_chunk_test = pd.DataFrame({
    'Id': chunk_ids,
    'Session': session,
    'Meeting': meeting,
    'Speaker' : speaker,
    'Text' : chunks
})

df_chunk_test.head()

,Id,Session,Meeting,Speaker,Text
0,S1M111,1,1,﻿Chairman,Excellencies. I am pleased to declare the firs...


In [14]:
# Initialize an empty DataFrame for chunks
df_chunk_test = pd.DataFrame()

# Iterate over DataFrame rows
for index, row in df_chunks.iterrows():
    chunks = chunk_text(row)
    session = [row['Session']] * len(chunks)
    meeting = [row['Meeting']] * len(chunks)
    speaker = [row['Speaker']] * len(chunks)
    id = row['Id']
    chunk_ids = [f'{id}{i}' for i in range(1, len(chunks) + 1)]

    df_temp = pd.DataFrame({
        'Id': chunk_ids,
        'Session': session,
        'Meeting': meeting,
        'Speaker' : speaker,
        'Text' : chunks
    })
    df_chunk_test = pd.concat([df_chunk_test, df_temp])

df_chunk_test.reset_index(drop=True, inplace=True)
df_chunk_test.head()

,Id,Session,Meeting,Speaker,Text
0,S1M111,1,1,﻿Chairman,Excellencies. I am pleased to declare the firs...
1,S1M121,1,1,Chile,Before going to the logistics and the backgrou...
2,S1M122,1,1,Chile,"weeks, I've heard several comments that take i..."
3,S1M123,1,1,Chile,entrance of the conference room. You will reme...
4,S1M131,1,1,Russia,"Thank you, Mr. President. I will now ask for t..."


In [15]:
df_chunk_test.tail()

,Id,Session,Meeting,Speaker,Text
1876,S3M7751,3,7,Russia,"Thank you very much, Chair. At the outset, Ple..."
1877,S3M7752,3,7,Russia,"talking about here However, it still remains u..."
1878,S3M7761,3,7,Chairman,I thank the distinguished representative of th...
1879,S3M7771,3,7,France,"Mercy, Mr. President. Thank you, Chair. I'd li..."
1880,S3M7781,3,7,Chairman,I think the distinguished representative of Fr...


In [16]:
#check how many rows
print(df_chunk_test.shape)

(1881, 5)


In [17]:
#check maximum length
max_length = df_chunks['Text'].str.len().max()
max_length

20322

In [18]:
#check maximum length
max_length = df_chunk_test['Text'].str.len().max()
max_length

2500

In [86]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [87]:
def generate_contextual_embedding(text):
    # Tokenize input text and enforce maximum sequence length
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    # Generate contextual embeddings
    with torch.no_grad():
        outputs = model(input_ids)
        # Extract contextual embeddings from the last layer
        last_hidden_states = outputs.last_hidden_state
    # Average pooling over tokens to get a single vector representation
    contextual_embedding = torch.mean(last_hidden_states, dim=1).squeeze().numpy()
    return contextual_embedding


In [26]:
#create a new column in the df_chunks with the embeddings and add the embeded each text
df_chunk_test['TextEmbeddings'] = df_chunk_test['Text'].apply(generate_contextual_embedding)

In [27]:
#create a new column in the df_chunks with the embeddings and add the embeded each text
df_chunk_test['SpeakerEmbeddings'] = df_chunk_test['Speaker'].apply(generate_contextual_embedding)

In [32]:
df_chunk_test.head()

,Id,Session,Meeting,Speaker,Text,TextEmbeddings,SpeakerEmbeddings
0,S1M111,1,1,﻿Chairman,Excellencies. I am pleased to declare the firs...,"[0.17065938, 0.08367448, 0.60545486, 0.1147223...","[0.35904002, 0.19308525, 0.9722154, 0.21520364..."
1,S1M121,1,1,Chile,Before going to the logistics and the backgrou...,"[-0.35539767, 0.3107825, 0.3086164, -0.1313286...","[0.35935688, 0.19950408, 0.97362196, 0.2157914..."
2,S1M122,1,1,Chile,"weeks, I've heard several comments that take i...","[-0.3258345, 0.1768665, 0.25171208, -0.1041317...","[0.35935688, 0.19950408, 0.97362196, 0.2157914..."
3,S1M123,1,1,Chile,entrance of the conference room. You will reme...,"[0.037276484, 0.08974554, 0.42191568, 0.032057...","[0.35935688, 0.19950408, 0.97362196, 0.2157914..."
4,S1M131,1,1,Russia,"Thank you, Mr. President. I will now ask for t...","[-0.26547337, 0.25839072, 0.3332973, -0.119123...","[0.3545946, 0.19809107, 0.9680495, 0.21662346,..."


In [31]:
print(df_chunk_test.shape)

(1881, 7)


In [34]:
#save df_chunks as a csv file
df_chunk_test.to_csv('all_meeting_emdeddings.csv', index=False)


In [40]:
import numpy as np

# Read each row into a dictionary and store in a list
rows = []
for i in range(len(df_chunk_test)):
    row = df_chunk_test.iloc[i].to_dict()
    # Convert ndarray to list if any
    for key, value in row.items():
        if isinstance(value, np.ndarray):
            row[key] = value.tolist()
    rows.append(row)

# Save in a json file
with open(f"{'all_meeting_emdeddings'}.json", "w") as f:
    json.dump(rows, f)

In [41]:

# Save in a json file
with open(f"{'all_meeting_emdeddings'}.json", "w") as f:
    json.dump(rows, f)

##Upload all meetings in vector base - Chroma


In [21]:
from azure.search.documents.indexes import SearchIndexClient
import requests
import json
import re
import pandas as pd

In [ ]:
#import cosmos db credentials
config = dotenv_values('credential.env')
ai_search_location = config['ai_search_location']
ai_search_key = config['ai_search_key']
ai_search_url = config['ai_search_url']
ai_search_index = 'oewg-speech-meeeting-index'
ai_search_name = 'aicpcigi'
embedding_length = 768

In [50]:
#read json

df = pd.read_json('/content/all_meeting_emdeddings.json')
df.head()

,Id,Session,Meeting,Speaker,Text,TextEmbeddings,SpeakerEmbeddings
0,S1M111,1,1,﻿Chairman,Excellencies. I am pleased to declare the firs...,"[0.17065937817096702, 0.08367448300123201, 0.6...","[0.359040021896362, 0.193085253238677, 0.97221..."
1,S1M121,1,1,Chile,Before going to the logistics and the backgrou...,"[-0.355397671461105, 0.310782492160797, 0.3086...","[0.359356880187988, 0.19950407743453902, 0.973..."
2,S1M122,1,1,Chile,"weeks, I've heard several comments that take i...","[-0.325834512710571, 0.176866501569747, 0.2517...","[0.359356880187988, 0.19950407743453902, 0.973..."
3,S1M123,1,1,Chile,entrance of the conference room. You will reme...,"[0.037276484072208, 0.089745536446571, 0.42191...","[0.359356880187988, 0.19950407743453902, 0.973..."
4,S1M131,1,1,Russia,"Thank you, Mr. President. I will now ask for t...","[-0.265473365783691, 0.258390724658966, 0.3332...","[0.354594588279724, 0.19809107482433302, 0.968..."


In [52]:
first_embedding_size = len(df['TextEmbeddings'][0])

print("Size of the first embedding vector:", first_embedding_size)

Size of the first embedding vector: 768


In [23]:
#connect to AI search
index_client = SearchIndexClient(endpoint=ai_search_url, credential=ai_search_key)

In [24]:
# Length of the embedding vector
EMBEDDING_LENGTH = 768

# Create index for AI Search with fields
# Note the datatypes for each field below
url = f"https://{ai_search_name}.search.windows.net/indexes/{ai_search_index}?api-version=2023-11-01"
payload = json.dumps(
    {
        "name": ai_search_index,
        "fields": [
            # Unique identifier for each document
            {
                "name": "id",
                "type": "Edm.String",
                "key": True,
                "filterable": True,
                "sortable": True,
                "retrievable": True,
            },
            # session name of the document
            {
                "name": "Session",
                "type": "Edm.String",
                "filterable": True,
                "sortable": True,
                "retrievable": True,
            },
            # meeting number of the document
            {
                "name": "Meeting",
                "type": "Edm.String",
                "filterable": True,
                "sortable": True,
                "retrievable": True,
            },
            # name of speaker
            {
                "name": "Speaker",
                "type": "Edm.String",
                "filterable": True,
                "searchable": True,
                "sortable": True,
                "retrievable": True,
            },
            #Speech Text
            {
                "name": "Text",
                "type": "Edm.String",
                "filterable": True,
                "sortable": True,
                "searchable": True,
                "retrievable": True,
            },
            # Vector embedding of the speaker
            {
                "name": "SpeakerEmbeddings",
                "type": "Collection(Edm.Single)",
                "searchable": True,
                "retrievable": True,
                "dimensions": EMBEDDING_LENGTH,
                "vectorSearchProfile": "vectorConfig",
            },
            # Vector embedding of the text content
            {
                "name": "TextEmbeddings",
                "type": "Collection(Edm.Single)",
                "searchable": True,
                "retrievable": True,
                "dimensions": EMBEDDING_LENGTH,
                "vectorSearchProfile": "vectorConfig",
            },
        ],
        "vectorSearch": {
            "algorithms": [{"name": "hnswConfig", "kind": "hnsw", "hnswParameters": {"metric": "cosine"}}],
            "profiles": [{"name": "vectorConfig", "algorithm": "hnswConfig"}],
        },
    }
)
headers = {"Content-Type": "application/json", "api-key": ai_search_key}

response = requests.request("PUT", url, headers=headers, data=payload)
if response.status_code == 201:
    print("Index created!")
elif response.status_code == 204:
    print("Index updated!")
else:
    print(f"HTTP request failed with status code {response.status_code}")
    print(f"HTTP response body: {response.text}")

Index created!


In [53]:
#check if there are dulplicate values in id column and print the duplicate rows
duplicate = df[df.duplicated(['Id'])]
duplicate

,Id,Session,Meeting,Speaker,Text,TextEmbeddings,SpeakerEmbeddings
1401,S4M111,4,2,Chairman,"Good afternoon. Excellencies, ladies and gentl...","[0.070496216416358, 0.10003080219030301, 0.516...","[0.359040021896362, 0.193085253238677, 0.97221..."
1402,S4M121,4,2,Malaysia,"Thank you, Mr Chair. At the outset, allow me t...","[-0.31548020243644703, 0.276718080043792, 0.34...","[0.361958622932434, 0.19766823947429601, 0.975..."
1403,S4M122,4,2,Malaysia,particularly the major players in outer space....,"[-0.171213015913963, 0.30578917264938305, 0.34...","[0.361958622932434, 0.19766823947429601, 0.975..."
1404,S4M131,4,2,Chairman,And I now give the floor to the distinguished ...,"[0.262150287628173, 0.099016658961772, 0.63854...","[0.359040021896362, 0.193085253238677, 0.97221..."
1405,S4M141,4,2,Peru,"Thank you, Chair. Let me begin by reiterating ...","[-0.29601949453353804, 0.264980763196945, 0.19...","[0.357784688472747, 0.19747737050056402, 0.972..."
...,...,...,...,...,...,...,...
1580,S4M1491,4,10,Germany,"Thank you, Mr. Chair, for giving me the floor ...","[-0.29199811816215504, 0.30838328599929804, 0....","[0.35806697607040405, 0.20071132481098103, 0.9..."
1581,S4M1501,4,10,Chairman,I thank the distinguished representative of Ge...,"[0.19157555699348403, 0.08949328958988101, 0.6...","[0.359040021896362, 0.193085253238677, 0.97221..."
1582,S4M1511,4,10,Sweden,"Thank you, Chair. I'll try to be brief. Sweden...","[0.012492157518863001, 0.242095723748207, 0.33...","[0.358288526535034, 0.202630922198295, 0.97252..."
1583,S4M1521,4,10,Chairman,I thank the distinguished representative of Sw...,"[0.206634223461151, 0.08458782732486701, 0.660...","[0.359040021896362, 0.193085253238677, 0.97221..."


In [57]:
#get indexes of duplicate values
duplicate_indexes = df[df['Id'].duplicated()].index

print("Indexes of duplicate values:", duplicate_indexes)

Indexes of duplicate values: Index([1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410,
       ...
       1574, 1575, 1576, 1578, 1579, 1580, 1581, 1582, 1583, 1584],
      dtype='int64', length=161)


In [59]:
duplicate_indexes_list = duplicate_indexes.tolist()

In [60]:
# replace values in at the indexes
for index in duplicate_indexes_list:
    old_id = df.at[index, 'Id']
    df.at[index, 'Id']  = f'{old_id}123'


In [61]:
#check if there are dulplicate values in id column and print the duplicate rows
duplicate = df[df.duplicated(['Id'])]
duplicate

,Id,Session,Meeting,Speaker,Text,TextEmbeddings,SpeakerEmbeddings
1468,S4M111123,4,9,Chairman,"Excellencies, distinguished delegates, ladies ...","[-0.154321834444999, 0.202155888080596, 0.3351...","[0.359040021896362, 0.193085253238677, 0.97221..."
1469,S4M121123,4,9,Russia,"Thank you very much, distinguished chair. You ...","[-0.06536568701267201, 0.181229948997497, 0.29...","[0.354594588279724, 0.19809107482433302, 0.968..."
1470,S4M131123,4,9,Chairman,"Before we continue with the list of speakers, ...","[0.013721786439418002, 0.14704015851020802, 0....","[0.359040021896362, 0.193085253238677, 0.97221..."
1471,S4M141123,4,9,Philippines,"Thank you Mr. Chair. First of all, thank you f...","[0.058499738574028, 0.18244618177413902, 0.390...","[0.35844904184341403, 0.19483409821987102, 0.9..."
1472,S4M151123,4,9,Chairman,I thank the distinguished representative of th...,"[0.18114757537841703, 0.07238890975713701, 0.6...","[0.359040021896362, 0.193085253238677, 0.97221..."
...,...,...,...,...,...,...,...
1575,S4M1451123,4,10,Chairman,"Thank you, Argentina, and I now call on the di...","[0.339551001787185, 0.08020855486392901, 0.666...","[0.359040021896362, 0.193085253238677, 0.97221..."
1576,S4M1461123,4,10,Mexico,"Thank you very much, Chairman. The Mexican del...","[-0.207748800516128, 0.186965823173522, 0.1748...","[0.361752152442932, 0.19620391726493802, 0.973..."
1578,S4M1471123,4,10,Chairman,Thank you. Mexico.,"[0.39111495018005304, 0.08668777346611001, 0.7...","[0.359040021896362, 0.193085253238677, 0.97221..."
1579,S4M1481123,4,10,Thailand,"Thank you, Mr. Chair. I will try to be brief. ...","[0.09575840830802901, 0.14930595457553802, 0.4...","[0.361303955316543, 0.19663545489311202, 0.973..."


In [68]:
#get indexes of duplicate values
duplicate_indexes = df[df['Id'].duplicated()].index

print("Indexes of duplicate values:", duplicate_indexes)

Indexes of duplicate values: Index([1523, 1524, 1525, 1526, 1528, 1529, 1530, 1531, 1532, 1533, 1535, 1536,
       1537, 1538, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549,
       1551, 1552, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1563, 1564,
       1566, 1567, 1568, 1569, 1570, 1571, 1573, 1574, 1575],
      dtype='int64')


In [69]:
duplicate_indexes_list = duplicate_indexes.tolist()
# replace values at the indexes
for i in range(len(duplicate_indexes_list)):
    index = duplicate_indexes_list[i]
    df.at[index, 'Id']  = f'S4M13{i}79'

In [70]:
#check if there are dulplicate values in id column and print the duplicate rows
duplicate = df[df.duplicated(['Id'])]
duplicate

,Id,Session,Meeting,Speaker,Text,TextEmbeddings,SpeakerEmbeddings


In [71]:
# Rename column 'old_name' to 'new_name'
df = df.rename(columns={'Id': 'id'})

In [74]:
# Read row by row and append to a list
data = []
for index, row in df.iterrows():
    row_dict = {}
    row_dict['id'] = row['id']
    row_dict['Session'] = str(row['Session'])
    row_dict['Meeting'] = str(row['Meeting'])
    row_dict['Speaker'] = row['Speaker']
    row_dict['Text'] = row['Text']
    row_dict['TextEmbeddings'] = row['TextEmbeddings']
    row_dict['SpeakerEmbeddings'] = row['SpeakerEmbeddings']
    row_dict['@search.action'] = 'upload'
    data.append(row_dict)


In [75]:

def insert_into_index(documents):
    """Uploads a list of 'documents' to Azure AI Search index."""

    url = f"https://{ai_search_name}.search.windows.net/indexes/{ai_search_index}/docs/index?api-version=2023-11-01"

    payload = json.dumps({"value": documents})
    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code == 200 or response.status_code == 201:
        return "Success"
    else:
        return f"Failure: {response.text}"

def make_safe_id(row_id: str):
    """Strips disallowed characters from row id for use as Azure AI search document ID."""
    return re.sub("[^0-9a-zA-Z_-]", "_", row_id)

#upload data in batchs of 1000
BATCH_SIZE = 1000
for i in range(0, len(data), BATCH_SIZE):
    batch = data[i:i+BATCH_SIZE]
    status = insert_into_index(batch)
    print(f'upload status: {status}')


upload status: Success
upload status: Success


##COnfigure AI search

In [79]:
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    SemanticField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)
import requests
import json
import re
import pandas as pd

In [80]:
#defining schema
index_client = SearchIndexClient(endpoint=ai_search_url, credential=ai_search_key)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True,
                filterable=True),
    SearchableField(name="Session", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True, filterable=True, sortable=True, facetable=True),
    SearchableField(name="Meeting", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True, filterable=True, sortable=True, facetable=True),
    SearchableField(name="Speaker", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True, filterable=True, sortable=True, facetable=True),
    SearchableField(name="Text", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True, filterable=True, sortable=True, facetable=True),
    SearchField(name="TextEmbeddings", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=768, vector_search_configuration="vector-config"),
    SearchField(name="SpeakerEmbeddings", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=768, vector_search_configuration="vector-config"),
]

In [81]:
#configure vector search
vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            })])

##Developing the RAG

In [113]:
#import libraries
import json
import pandas as pd
import requests
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryType,
    VectorizedQuery
)

from pprint import pprint
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

In [94]:
search_client = SearchClient(ai_search_url, ai_search_index, AzureKeyCredential(ai_search_key))

In [88]:
#test index retrival
question = 'what is iran stand?'
question_embedding = generate_contextual_embedding(question)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [97]:
def retrieve_vector_top_chunks(k, question_embedding):
    """Retrieve the top K entries from Azure AI Search using vector search with speaker embedding."""
    vector_query = VectorizedQuery(vector=question_embedding,
                                k_nearest_neighbors=k,
                                fields="SpeakerEmbeddings")

    results = search_client.search(
        search_text=None,
        vector_queries=[vector_query],
        select=["Speaker", "Text","Meeting", "Session"],
        top=k
    )
    output = [[f'Session: {result["Session"]}',f'Meeting: {result["Meeting"]}',f'{result["Speaker"]}: {result["Text"]}'] for result in results]

    return output


In [98]:
#trying only vector search
results = retrieve_vector_top_chunks(3, question_embedding)
for result in results:
    print(result)

['Session: 4', 'Meeting: 10', 'Philippines (Multi Country): I have the honor to take the floor on behalf of Argentina, Australia, Austria, Belgium, Brazil, Canada, Chile, Costa Rica, Colombia, Czech Republic, Denmark, Ecuador, El Salvador, Finland, France, Germany, Greece, Ireland, Italy, Japan, Luxembourg, Malawi. Mexico, the Netherlands, New Zealand, Nigeria, Norway, Panama, Peru, my country the Philippines, Portugal, the Republic of Korea, Samoa, Singapore, Spain, Sweden, Switzerland, the United Kingdom, and Uruguay. While we could not adopt a substantive report in accordance with the mandate of the Open Ended Working Group on reducing space threats through norms, rules, and principles of responsible behavior, we nevertheless thank you and your team for the excellent, fair, and balanced manner with which we have led this process. We welcome that 34 states delivered a joint statement in support of the OEWG. which has now been submitted officially as a working paper. We reiterate our 

In [102]:
def retrieve_hybrid_top_chunks(k, question, question_embedding):
    """Retrieve the top K entries from Azure AI Search using hybrid search with speaker embedding."""
    vector_query = VectorizedQuery(vector=question_embedding,
                                k_nearest_neighbors=k,
                                fields="TextEmbeddings")

    results = search_client.search(
        search_text=question,
        vector_queries=[vector_query],
        select=["Speaker", "Text","Meeting", "Session"],
        top=k
    )

    output = [[f'Session: {result["Session"]}',f'Meeting: {result["Meeting"]}',f'{result["Speaker"]}: {result["Text"]}'] for result in results]

    return output

In [105]:
results = retrieve_hybrid_top_chunks(3, question, question_embedding)
results

[['Session: 1',
  'Meeting: 4',
  "Iran: Thank you, Mr. President. In this occasion, I would like to congratulate you on taking over the leadership of this open-ended working team, as we thank the members of the discussion panel today. The Islamic Republic of Iran confirms its position that the external space is the and the international community. And the exploration and use of foreign resources must be for the benefit of all countries or nations, and foreign resources must be a field for international peace and cooperation. Mr. President, the international security, economic development and prosperity of countries depend more on foreign countries. These two similar directions have become more common, and that is to implement the protection of foreign countries and to implement protection of foreign countries from the effects of of the political agenda. We must also distinguish between the responsible and non-responsible behavior in the external space. This will strengthen the agenda 

In [107]:
#using hybrid context because it's search is more related
def get_hybrid_context(user_question, retrieved_k = 5):
    # Generate embeddings for the question
    question_embedding = generate_contextual_embedding(user_question)

    # Retrieve the top K entries
    output = retrieve_hybrid_top_chunks(retrieved_k, user_question, question_embedding)

    # concatenate the content of the retrieved documents
    context = '. '.join([item for sublist in output for item in sublist])

    return context

In [108]:
context = get_hybrid_context('iran', retrieved_k = 5)
context

"Session: 1. Meeting: 4. Iran: Thank you, Mr. President. In this occasion, I would like to congratulate you on taking over the leadership of this open-ended working team, as we thank the members of the discussion panel today. The Islamic Republic of Iran confirms its position that the external space is the and the international community. And the exploration and use of foreign resources must be for the benefit of all countries or nations, and foreign resources must be a field for international peace and cooperation. Mr. President, the international security, economic development and prosperity of countries depend more on foreign countries. These two similar directions have become more common, and that is to implement the protection of foreign countries and to implement protection of foreign countries from the effects of of the political agenda. We must also distinguish between the responsible and non-responsible behavior in the external space. This will strengthen the agenda of the wor

In [110]:

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)


In [116]:
query = "what is iran's point?"
context = get_hybrid_context('iran', retrieved_k = 5)

In [114]:
def format_query(query, context):
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

In [117]:
# format the query in the form generator expects the input
query = format_query(query, context)
pprint(query)

("question: what is iran's point? context: Session: 4. Meeting: 10. Iran: "
 'Thank you, Mr. Chairman. Let me to join previous speakers in thanking you '
 'and your team for excellent work you have done on guiding us through this '
 'two-year journey in open-ended working group on reducing threat in outer '
 'space. Also, let me to thank the Secretariat and distinguished panelists for '
 'their contribution in this process. Their insightful contributions enriched '
 'the discussion through the work that we have done in this process. Despite '
 'the fact that we casted our vote against the resolution that established '
 'this working group, my delegation entered this process with open mind and '
 'constructive approach. We clearly elaborated our concern and our expectation '
 'from this exercise with the aim of making it a success. It is clear that '
 'member states have different perceptions from the threat that emanates from '
 'space and space technologies. Space technology should co

In [118]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [119]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('Iran is a nuclear power. They have the capability to build a nuclear bomb, '
 "but they don't want to because they don't want the world to know about it. "
 "They don't want")


In [120]:
query = "what is canada's point?"
context = get_hybrid_context(query, retrieved_k = 5)
query = format_query(query, context)
generate_answer(query)

('Canada has a long history of being a big player in the international space '
 'race. They are the only country to have launched a satellite into orbit, and '
 'they are the only country to have launched')
